# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [68]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [69]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/matthew/Code/Udacity/2.5.0_Data_Modeling_with_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [70]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [71]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create Cassandra Tables and process event_datafile_new.csv to load the data. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The example image below is a screenshot of what the denormalized data looks like in event_datafile_new.csv.

<img src="images/image_event_datafile_new.jpg">

#### Connect to Cassandra Cluster, create keyspace and set keyspace

In [72]:
# Make a connection to a Cassandra instance on local machine 
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #Locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)   
    
# Set KEYSPACE
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

try:
    session.set_keyspace('sparkify')
    session.row_factory = pandas_factory
except Exception as e:
    print(e)   

## Three Queries to answer the following questions

### Query 1. List the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

`Table Name: sessions
column 1: session_id
column 2: item_in_session
column 3: artist
column 4: song
column 5: length
PRIMARY KEY(session_id, item_in_session)`

#### Query 1. Create Table sessions

In [73]:
query = "DROP TABLE IF EXISTS sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(session_id INT, item_in_session INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Query 1. Load Data into Table

In [74]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO sessions (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Query 1 Verify the table data
`SELECT artist, song, length FROM sessions WHERE session_id=338 AND item_in_session=4;`

In [75]:
query = "SELECT artist, song, length FROM sessions WHERE session_id=338 AND item_in_session=4;"
try:
    rows = session.execute(query)
    df = rows._current_rows[['artist', 'song', 'length']]
except Exception as e:
    print(e)
    
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2. name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
   
`Table Name: user_sessions
column 1: user_id
column 2: session_id
column 3: item_in_session
column 4: artist
column 5: song
column 6: first_name
column 7: last_name
PRIMARY KEY(user_id, session_id, item_in_session)`

Here the Primary Key has three fields: user_id and session_id are the partition key, and item_in_session are clustering keys. 
Partitioning is done by user_id and session_id, and within that partition, rows are ordered by the item_in_session.

#### Query 2. Create Table user_sessions

In [76]:
query = "DROP TABLE IF EXISTS user_sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_sessions "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, first_name TEXT, last_name TEXT, PRIMARY KEY ((user_id), session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [77]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Query 2 Verify the table data
`SELECT artist, song, first_name, last_name from user_sessions WHERE user_id=10 and session_id=182;`

In [78]:
query = "SELECT artist, song, first_name, last_name from user_sessions WHERE user_id=10 and session_id=182;"
try:
    rows = session.execute(query)
    df = rows._current_rows[['artist', 'song', 'first_name', 'last_name']]
except Exception as e:
    print(e)
    
df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

`Table Name: user_songs
column 1: song
column 2: user_id
column 3: first_name
column 4: last_name
PRIMARY KEY(song, user_id)`

Here the Primary Key has two fields: song is the partition key, and userid are clustering keys. 
Partitioning is done by song and within that partition, rows are ordered by the userid.

#### Query 3. Create Table user_songs

In [79]:
query = "DROP TABLE IF EXISTS user_songs"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(song TEXT, user_id INT, first_name TEXT, last_name TEXT, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [80]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Query 3 Verify the table data

In [81]:
query = "select first_name, last_name from user_songs WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    df = rows._current_rows[['first_name', 'last_name']]
except Exception as e:
    print(e)
    
df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [82]:
query = "DROP TABLE IF EXISTS sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_songs"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [83]:
session.shutdown()
cluster.shutdown()